<a href="https://colab.research.google.com/github/gabriel-hc-dev/duvida-zero-alura/blob/main/projetoAluraGabriel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ==============================================================================
# PROJETO IMERSÃO IA ALURA: AGENTE "DÚVIDA ZERO"
# ==============================================================================

!pip -q install google-genai



# Importa as bibliotecas necessárias
import os
from google.colab import userdata
import google.generativeai as genai
from IPython.display import display, Markdown
import textwrap
import re # Para expressões regulares

# Configura a API Key do Google Gemini
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

# Configura o cliente da SDK do Gemini
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

# Define o ID do modelo a ser utilizado.
MODEL_ID = "gemini-2.0-flash"
# Instancia o modelo para uso geral
model = genai.GenerativeModel(MODEL_ID)

# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

# ==============================================================================
# AGENTE PRINCIPAL: DESMISTIFICADOR DE TEXTO
# ==============================================================================

def desmistificar_texto(texto_complexo):
    """
    Função principal do Agente 'Dúvida Zero'.
    Recebe um texto complexo e retorna uma versão simplificada ou explicações de termos.
    """
    prompt = f"""
    Você é um especialista em simplificar informações complexas para qualquer público.
    Sua tarefa é analisar o texto a seguir, identificar termos ou frases que podem ser difíceis
    de entender para uma pessoa leiga e, em seguida, explicá-los de forma clara e concisa.
    Texto complexo a ser desmistificado:
    "{texto_complexo}"

    Por favor, apresente as explicações e/ou a versão simplificada de forma didática e fácil de compreender, utilizando a formatação Markdown."""

    try:
        response = model.generate_content(prompt)
        return response.text
    except Exception as e:
        return f"Ocorreu um erro ao processar o texto com o agente principal: {e}"

# ==============================================================================
# AGENTES AUXILIARES
# ==============================================================================

# --- Agente Auxiliar 1: Validador de Contexto ---
def validar_contexto_e_classificar(texto):
    """
    Avalia se o texto possui contexto suficiente e tenta classificar sua área.
    Retorna uma tupla (contexto_status, categoria).
    contexto_status: "completo", "incompleto", "curto_demais"
    categoria: "jurídico", "científico", "financeiro", "tecnológico", "filosófico", "geral", "desconhecido"
    """
    if len(texto.strip()) < 20: # Exemplo: texto muito curto
        return "curto_demais", "desconhecido"

    prompt_contexto = f"""Analise o seguinte texto para determinar se ele é autoexplicativo (contexto completo)
    ou se parece faltar informações para sua total compreensão (contexto incompleto).
    Além disso, classifique a área principal do texto em uma das seguintes categorias:
    'jurídico', 'científico', 'financeiro', 'tecnológico', 'filosófico', 'geral'.

    Formato de saída esperado (APENAS ESTE FORMATO):
    Contexto: [completo/incompleto]
    Categoria: [jurídico/científico/financeiro/tecnológico/filosófico/geral/desconhecido]

Texto:
"{texto}"
"""
    try:
        response = model.generate_content(prompt_contexto)
        # O modelo pode adicionar texto extra, então precisamos parsear a resposta
        lines = response.text.strip().split('\n')
        context_status = "desconhecido"
        category = "desconhecido"

        for line in lines:
            if "Contexto:" in line:
                context_status = line.split("Contexto:")[1].strip().lower()
                context_status = context_status.replace('[', '').replace(']', '') # Remove colchetes se houver
            if "Categoria:" in line:
                category = line.split("Categoria:")[1].strip().lower()
                category = category.replace('[', '').replace(']', '') # Remove colchetes se houver

        return context_status, category
    except Exception as e:
        print(f"Erro no Agente Validador de Contexto: {e}")
        return "erro", "desconhecido"

# --- Agente Auxiliar 2: Perguntador Inteligente ---
def gerar_pergunta_esclarecimento(texto_original, problema_identificado="geral"):
    """
    Gera uma pergunta para o usuário para obter mais contexto ou esclarecimento.
    """
    prompt_pergunta = f"""Você é um assistente atencioso que ajuda a obter mais informações.
Com base no texto original a seguir e no problema identificado (se houver),
formule uma pergunta clara e concisa para o usuário que o ajude a fornecer mais contexto ou detalhe.
Se o problema for 'curto_demais', peça mais texto.
Se o problema for 'incompleto', peça informações específicas que pareçam faltar.

Texto original: "{texto_original}"
Problema identificado: {problema_identificado}

Por favor, gere apenas a pergunta. Seja direto e útil."""

    try:
        response = model.generate_content(prompt_pergunta)
        return response.text
    except Exception as e:
        return f"Ocorreu um erro ao gerar a pergunta: {e}"

# --- Agente Auxiliar 3: Verificador de Sinônimos/Alternativas ---
def sugerir_sinonimos_simples(termo, contexto_texto):
    """
    Sugere sinônimos ou frases alternativas mais simples para um termo específico,
    considerando o contexto do texto original.
    """
    prompt_sinonimo = f"""Para o termo "**{termo}**" presente no contexto do texto:
"{contexto_texto}"

Por favor, forneça 3 sinônimos ou frases alternativas mais simples que possam
substituí-lo sem alterar o sentido do contexto. Apresente-os em uma lista Markdown.
"""
    try:
        response = model.generate_content(prompt_sinonimo)
        return response.text
    except Exception as e:
        return f"Ocorreu um erro ao sugerir sinônimos: {e}"


# ==============================================================================
# LOOP PRINCIPAL DO AGENTE "DÚVIDA ZERO" COM INTERAÇÃO
# ==============================================================================

print("Seja bem-vindo ao Agente 'Dúvida Zero'! 🚀")
print("Este agente irá desmistificar textos complexos para você, usando IA.")
print("💡 Dica: Experimente textos jurídicos, científicos, econômicos ou tecnológicos.")
print("---")
print("Digite 'sair' a qualquer momento para encerrar.")

while True:
    print("\n" + "=" * 80) # Separador visual
    texto_input = input("👉 Cole o texto que você quer desmistificar (ou 'sair'):\n")

    if texto_input.lower().strip() == 'sair':
        print("\n👋 Obrigado por usar o Agente 'Dúvida Zero'. Até a próxima!")
        break

    if not texto_input.strip():
        print("❌ Por favor, cole um texto válido antes de prosseguir.")
        continue

    # --- Etapa 1: Validação e Classificação de Contexto ---
    print("\n⏳ Avaliando o contexto do texto...")
    context_status, category = validar_contexto_e_classificar(texto_input)
    print(f"✔️ Contexto avaliado: **{context_status.capitalize()}** | Categoria: **{category.capitalize()}**")

    # --- Etapa 2: Tomada de Decisão com Base no Contexto ---
    if context_status == "curto_demais":
        print("\n🤔 Seu texto parece ser muito curto. Não consigo desmistificar algo tão breve.")
        pergunta = gerar_pergunta_esclarecimento(texto_input, "curto_demais")
        print(f"\n{pergunta}")
        continue # Volta para o início do loop para nova entrada

    elif context_status == "incompleto":
        print("\n🧐 Seu texto parece incompleto ou falta contexto para uma boa desmistificação.")
        pergunta = gerar_pergunta_esclarecimento(texto_input, "incompleto")
        print(f"\n{pergunta}")
        # Aqui você poderia, opcionalmente, pedir um novo input do usuário para complementar
        # Ou simplesmente continuar com a desmistificação sabendo que pode não ser ideal
        # Por simplicidade, vamos prosseguir com a desmistificação.
        print("\n⚠️ Tentarei desmistificar com o que tenho, mas o resultado pode não ser ideal.")

    # --- Etapa 3: Desmistificação Principal ---
    print("\n⏳ Processando seu texto com o Agente Desmistificador...")
    resposta_agente_principal = desmistificar_texto(texto_input)
    print("\n--- Resultado do Agente ---")
    display(to_markdown(resposta_agente_principal))

    # --- Etapa 4: Opção para Sugerir Sinônimos ---
    print("\n--- Outras Opções ---")
    quer_sinonimos = input("Deseja sugerir sinônimos para um termo específico? (s/n): ").lower().strip()
    if quer_sinonimos == 's':
        termo_sinonimo = input("Qual termo do texto original você gostaria de explorar sinônimos mais simples? ").strip()
        if termo_sinonimo:
            print(f"\n⏳ Sugerindo sinônimos para '{termo_sinonimo}'...")
            sinonimos = sugerir_sinonimos_simples(termo_sinonimo, texto_input)
            print("\n--- Sugestões de Sinônimos ---")
            display(to_markdown(sinonimos))
        else:
            print("Nenhum termo fornecido para sinônimos.")

    print("\n" + "=" * 80 + "\n") # Separador visual

Seja bem-vindo ao Agente 'Dúvida Zero'! 🚀
Este agente irá desmistificar textos complexos para você, usando IA.
💡 Dica: Experimente textos jurídicos, científicos, econômicos ou tecnológicos.
---
Digite 'sair' a qualquer momento para encerrar.

